# Latent Dirichlet Allocation (LDA)

In [3]:
# 親のフォルダのパスを追加
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [4]:
# Movielensのデータの読み込み
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [8]:
import gensim
import logging
from gensim.corpora.dictionary import Dictionary

movie_content = movielens.item_content.copy()
# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:list(map(str, x)))

# タグとジャンルデータを使って、LDAを学習する
tag_genre_data = movie_content.tag_genre.tolist()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
common_dictionary = Dictionary(tag_genre_data)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_genre_data]

# LDAの学習
lda_model = gensim.models.LdaModel(common_corpus, id2word=common_dictionary, num_topics=50, passes=30)
lda_topics = lda_model[common_corpus]




2021-08-24 02:38:24,041 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-08-24 02:38:24,219 : INFO : adding document #10000 to Dictionary(14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...)
2021-08-24 02:38:24,230 : INFO : built Dictionary(15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...) from 10681 documents (total 117144 corpus positions)
2021-08-24 02:38:24,230 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...) from 10681 documents (total 117144 corpus positions)", 'datetime': '2021-08-24T02:38:24.230595', 'gensim': '4.0.1', 'python': '3.7.8 (default, May  2 2021, 19:43:55) \n[Clang 10.0.1 (clang-1001.0.46.4)]', 'platform': 'Darwin-18.7.0-x86_64-i386-64bit', 'event': 'created'}
2021-08-24 02:38:24,321 : INFO : using symmetric alpha at 0.02
2021-08-24 02:38:24,322 : INFO : using symmetric eta at 0.02
2021-08-24 02:38:2

2021-08-24 02:38:26,361 : INFO : topic #46 (0.020): 0.025*"natalie portman" + 0.021*"kevin spacey" + 0.018*"Drama" + 0.016*"sci-fi" + 0.014*"ridley scott" + 0.014*"ewan mcgregor" + 0.013*"must see!" + 0.012*"steven spielberg" + 0.010*"recorded" + 0.010*"moving"
2021-08-24 02:38:26,362 : INFO : topic diff=0.235645, rho=0.447214
2021-08-24 02:38:26,531 : INFO : -21.546 per-word bound, 3061396.5 perplexity estimate based on a held-out corpus of 681 documents with 5199 words
2021-08-24 02:38:26,531 : INFO : PROGRESS: pass 0, at document #10681/10681
2021-08-24 02:38:26,638 : INFO : merging changes from 681 documents into a model of 10681 documents
2021-08-24 02:38:26,664 : INFO : topic #13 (0.020): 0.071*"time travel" + 0.050*"twist ending" + 0.045*"holocaust" + 0.037*"post-apocalyptic" + 0.027*"bill murray" + 0.022*"good dialogue" + 0.016*"capitalism" + 0.015*"michael moore" + 0.014*"singing" + 0.014*"twist"
2021-08-24 02:38:26,664 : INFO : topic #47 (0.020): 0.250*"less than 300 ratings"

2021-08-24 02:38:28,238 : INFO : topic #22 (0.020): 0.036*"philip k. dick" + 0.036*"oscar (best cinematography)" + 0.032*"journalism" + 0.031*"angelina jolie" + 0.030*"based on a book" + 0.029*"classic" + 0.026*"cult film" + 0.022*"earnest" + 0.020*"adapted from:book" + 0.019*"political"
2021-08-24 02:38:28,239 : INFO : topic #40 (0.020): 0.122*"magic" + 0.082*"nudity (full frontal - notable)" + 0.057*"Drama" + 0.054*"edward norton" + 0.051*"sports" + 0.035*"not corv lib" + 0.033*"hw drama" + 0.030*"baseball" + 0.025*"based on a book" + 0.025*"don cheadle"
2021-08-24 02:38:28,240 : INFO : topic #35 (0.020): 0.137*"comedy" + 0.092*"Comedy" + 0.088*"funny" + 0.044*"ummarti2006" + 0.039*"parody" + 0.024*"hilarious" + 0.021*"Romance" + 0.020*"dani2006" + 0.020*"afi 100 (laughs)" + 0.019*"spoof"
2021-08-24 02:38:28,241 : INFO : topic #48 (0.020): 0.042*"surreal" + 0.040*"quirky" + 0.029*"Drama" + 0.027*"bittersweet" + 0.025*"Comedy" + 0.022*"whimsical" + 0.020*"humorous" + 0.020*"woody alle

2021-08-24 02:38:29,798 : INFO : topic #29 (0.020): 0.099*"action" + 0.047*"adventure" + 0.035*"sequel" + 0.034*"Action" + 0.031*"pirates" + 0.029*"arnold schwarzenegger" + 0.026*"robots" + 0.026*"motorcycle" + 0.020*"Adventure" + 0.019*"death"
2021-08-24 02:38:29,799 : INFO : topic #19 (0.020): 0.065*"ghosts" + 0.054*"pg-13" + 0.034*"wedding" + 0.033*"jude law" + 0.032*"julia roberts" + 0.030*"sam peckinpah" + 0.023*"want to own" + 0.022*"prostitution" + 0.020*"infidelity" + 0.019*"hollywood"
2021-08-24 02:38:29,800 : INFO : topic #36 (0.020): 0.083*"dark comedy" + 0.066*"black comedy" + 0.061*"bechdel test:fail" + 0.039*"funniest movies" + 0.031*"cold war" + 0.028*"nazis" + 0.019*"classic" + 0.017*"Comedy" + 0.014*"russia" + 0.014*"seen more than once"
2021-08-24 02:38:29,801 : INFO : topic diff=0.124645, rho=0.346261
2021-08-24 02:38:29,803 : INFO : PROGRESS: pass 2, at document #10000/10681
2021-08-24 02:38:30,093 : INFO : merging changes from 2000 documents into a model of 10681 d

2021-08-24 02:38:31,414 : INFO : topic diff=0.118591, rho=0.327201
2021-08-24 02:38:31,416 : INFO : PROGRESS: pass 3, at document #8000/10681
2021-08-24 02:38:31,681 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:38:31,710 : INFO : topic #48 (0.020): 0.054*"quirky" + 0.050*"surreal" + 0.028*"bittersweet" + 0.026*"woody allen" + 0.023*"humorous" + 0.021*"Comedy" + 0.021*"peter jackson" + 0.020*"whimsical" + 0.019*"coming of age" + 0.018*"fantasy"
2021-08-24 02:38:31,710 : INFO : topic #17 (0.020): 0.062*"police" + 0.050*"will smith" + 0.033*"computers" + 0.023*"corruption" + 0.019*"no dialogue" + 0.018*"small-town life" + 0.017*"milla jovovich" + 0.016*"stage" + 0.016*"writer" + 0.015*"race"
2021-08-24 02:38:31,711 : INFO : topic #12 (0.020): 0.097*"quentin tarantino" + 0.096*"christmas" + 0.068*"heist" + 0.065*"tarantino" + 0.048*"nonlinear" + 0.042*"rape" + 0.023*"ireland" + 0.019*"blindfold" + 0.018*"xmas theme" + 0.016*"terry gilliam"
2021

...

2021-08-24 02:39:09,534 : INFO : topic #17 (0.020): 0.062*"police" + 0.056*"will smith" + 0.031*"computers" + 0.029*"corruption" + 0.027*"sandra bullock" + 0.019*"milla jovovich" + 0.019*"stage" + 0.018*"no dialogue" + 0.018*"torture" + 0.016*"action packed"
2021-08-24 02:39:09,535 : INFO : topic #6 (0.020): 0.091*"martial arts" + 0.058*"cult film" + 0.042*"gothic" + 0.035*"campy" + 0.026*"cult classic" + 0.026*"silly" + 0.025*"australia" + 0.024*"australian" + 0.021*"sam raimi" + 0.020*"seen 2006"
2021-08-24 02:39:09,536 : INFO : topic diff=0.040500, rho=0.178627
2021-08-24 02:39:09,538 : INFO : PROGRESS: pass 25, at document #10000/10681
2021-08-24 02:39:09,785 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:39:09,812 : INFO : topic #34 (0.020): 0.173*"War" + 0.126*"Drama" + 0.092*"world war ii" + 0.069*"drugs" + 0.050*"war" + 0.041*"history" + 0.033*"Action" + 0.029*"prison" + 0.025*"london" + 0.019*"Adventure"
2021-08-24 02:39:09,813 : INF

2021-08-24 02:39:10,991 : INFO : topic diff=0.048196, rho=0.175844
2021-08-24 02:39:10,992 : INFO : PROGRESS: pass 26, at document #8000/10681
2021-08-24 02:39:11,230 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:39:11,257 : INFO : topic #20 (0.020): 0.135*"politics" + 0.089*"documentary" + 0.065*"jack nicholson" + 0.043*"terrorism" + 0.037*"seen 2008" + 0.034*"propaganda" + 0.032*"Documentary" + 0.028*"political" + 0.028*"love story" + 0.027*"aviation"
2021-08-24 02:39:11,258 : INFO : topic #47 (0.020): 0.292*"less than 300 ratings" + 0.147*"Drama" + 0.062*"nudity (rear)" + 0.037*"jackie chan" + 0.025*"kung fu" + 0.023*"football" + 0.017*"oppl" + 0.016*"brian de palma" + 0.016*"denzel washington" + 0.013*"bechdel test:pass"
2021-08-24 02:39:11,259 : INFO : topic #18 (0.020): 0.050*"creepy" + 0.046*"franchise" + 0.045*"surrealism" + 0.036*"slasher" + 0.027*"lurid" + 0.024*"robert downey jr" + 0.022*"united states" + 0.022*"adolescence" + 0.0

2021-08-24 02:39:12,652 : INFO : topic #35 (0.020): 0.171*"comedy" + 0.132*"Comedy" + 0.081*"funny" + 0.052*"parody" + 0.039*"hilarious" + 0.035*"can't remember" + 0.023*"stupid" + 0.022*"spoof" + 0.020*"funny as hell" + 0.018*"seen more than once"
2021-08-24 02:39:12,653 : INFO : topic #49 (0.020): 0.103*"anime" + 0.063*"psychology" + 0.056*"serial killer" + 0.053*"teen" + 0.050*"high school" + 0.019*"mental illness" + 0.018*"japan" + 0.017*"cannibalism" + 0.016*"anthony hopkins" + 0.014*"cult film"
2021-08-24 02:39:12,654 : INFO : topic #8 (0.020): 0.102*"space" + 0.044*"harrison ford" + 0.026*"christianity" + 0.024*"george lucas" + 0.024*"kevin smith" + 0.024*"spaghetti western" + 0.023*"archaeology" + 0.020*"sergio leone" + 0.019*"fantasy" + 0.019*"space opera"
2021-08-24 02:39:12,655 : INFO : topic #46 (0.020): 0.091*"dvd" + 0.063*"overrated" + 0.055*"seen more than once" + 0.042*"owned" + 0.031*"samuel l. jackson" + 0.030*"bibliothek" + 0.030*"imdb top 250" + 0.029*"eric's dvds" 

2021-08-24 02:39:14,166 : INFO : topic #49 (0.020): 0.088*"anime" + 0.057*"serial killer" + 0.057*"psychology" + 0.057*"teen" + 0.057*"high school" + 0.020*"mental illness" + 0.017*"cannibalism" + 0.016*"cult film" + 0.015*"japan" + 0.014*"anthony hopkins"
2021-08-24 02:39:14,167 : INFO : topic #48 (0.020): 0.048*"quirky" + 0.044*"directorial debut" + 0.044*"surreal" + 0.026*"woody allen" + 0.025*"bittersweet" + 0.024*"humorous" + 0.021*"underrated" + 0.020*"coming of age" + 0.019*"erlend's dvds" + 0.018*"whimsical"
2021-08-24 02:39:14,168 : INFO : topic diff=0.022702, rho=0.170646
2021-08-24 02:39:14,169 : INFO : PROGRESS: pass 28, at document #6000/10681
2021-08-24 02:39:14,396 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:39:14,422 : INFO : topic #45 (0.020): 0.044*"tense" + 0.042*"japan" + 0.041*"clint eastwood" + 0.032*"forceful" + 0.032*"revenge" + 0.030*"gritty" + 0.029*"atmospheric" + 0.024*"Action" + 0.023*"sweeping" + 0.022*"samura

2021-08-24 02:39:15,556 : INFO : topic diff=0.131329, rho=0.168215
2021-08-24 02:39:15,557 : INFO : PROGRESS: pass 29, at document #4000/10681
2021-08-24 02:39:15,799 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:39:15,824 : INFO : topic #0 (0.020): 0.097*"crime" + 0.061*"brad pitt" + 0.060*"violence" + 0.043*"Crime" + 0.038*"morgan freeman" + 0.037*"violent" + 0.032*"organized crime" + 0.030*"thriller" + 0.028*"drama" + 0.027*"philip seymour hoffman"
2021-08-24 02:39:15,825 : INFO : topic #26 (0.020): 0.225*"classic" + 0.147*"oscar (best picture)" + 0.059*"oscar (best directing)" + 0.046*"oscar (best supporting actor)" + 0.035*"afi 100" + 0.031*"afi 100 (laughs)" + 0.028*"national film registry" + 0.022*"afi 100 (cheers)" + 0.021*"matt damon" + 0.018*"oscar (best supporting actress)"
2021-08-24 02:39:15,826 : INFO : topic #7 (0.020): 0.178*"movie to see" + 0.071*"mel gibson" + 0.053*"boxing" + 0.034*"suburbia" + 0.031*"uma thurman" + 0.029*

In [9]:
# LDAContentレコメンド
from src.lda_content import LDAContentRecommender
recommender = LDAContentRecommender()
recommend_result = recommender.recommend(movielens)

2021-08-24 02:42:09,772 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-08-24 02:42:09,943 : INFO : adding document #10000 to Dictionary(14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...)
2021-08-24 02:42:09,953 : INFO : built Dictionary(15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...) from 10681 documents (total 117144 corpus positions)
2021-08-24 02:42:09,954 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...) from 10681 documents (total 117144 corpus positions)", 'datetime': '2021-08-24T02:42:09.954077', 'gensim': '4.0.1', 'python': '3.7.8 (default, May  2 2021, 19:43:55) \n[Clang 10.0.1 (clang-1001.0.46.4)]', 'platform': 'Darwin-18.7.0-x86_64-i386-64bit', 'event': 'created'}
2021-08-24 02:42:10,040 : INFO : using symmetric alpha at 0.02
2021-08-24 02:42:10,040 : INFO : using symmetric eta at 0.02
2021-08-24 02:42:1

2021-08-24 02:42:11,985 : INFO : topic #16 (0.020): 0.080*"nudity (rear)" + 0.053*"pg" + 0.041*"military" + 0.035*"claymation" + 0.031*"weird" + 0.030*"nudity (topless)" + 0.024*"aardman" + 0.019*"nudity (topless - brief)" + 0.019*"childhood" + 0.018*"penguins"
2021-08-24 02:42:11,986 : INFO : topic diff=0.241935, rho=0.447214
2021-08-24 02:42:12,143 : INFO : -21.496 per-word bound, 2957512.5 perplexity estimate based on a held-out corpus of 681 documents with 5199 words
2021-08-24 02:42:12,144 : INFO : PROGRESS: pass 0, at document #10681/10681
2021-08-24 02:42:12,246 : INFO : merging changes from 681 documents into a model of 10681 documents
2021-08-24 02:42:12,272 : INFO : topic #36 (0.020): 0.035*"russell crowe" + 0.031*"virus" + 0.029*"not available from netflix" + 0.028*"Comedy" + 0.026*"based on a book" + 0.025*"cult film" + 0.025*"peter jackson" + 0.024*"movielens quickpick" + 0.023*"1980s" + 0.021*"adam sandler"
2021-08-24 02:42:12,273 : INFO : topic #6 (0.020): 0.125*"Musical

2021-08-24 02:42:13,783 : INFO : topic #42 (0.020): 0.086*"martial arts" + 0.059*"robin williams" + 0.042*"jonossa" + 0.041*"seen 2006" + 0.031*"japan" + 0.029*"samurai" + 0.027*"Drama" + 0.025*"orson welles" + 0.024*"hugh grant" + 0.023*"divorce"
2021-08-24 02:42:13,784 : INFO : topic #41 (0.020): 0.141*"remake" + 0.058*"new york city" + 0.049*"christmas" + 0.044*"family" + 0.037*"new york" + 0.031*"Drama" + 0.027*"seen" + 0.024*"kevin spacey" + 0.023*"predictable" + 0.021*"friday night movie"
2021-08-24 02:42:13,784 : INFO : topic #5 (0.020): 0.162*"classic" + 0.082*"musical" + 0.027*"afi 100 (laughs)" + 0.024*"los angeles" + 0.018*"witch" + 0.018*"national film registry" + 0.016*"john travolta" + 0.015*"Musical" + 0.014*"adapted from b'way" + 0.013*"movie to see"
2021-08-24 02:42:13,785 : INFO : topic #49 (0.020): 0.081*"national film registry" + 0.026*"natalie portman" + 0.025*"tumey's dvds" + 0.022*"Film-Noir" + 0.020*"film noir" + 0.020*"black and white" + 0.019*"imdb top 250" + 

2021-08-24 02:42:15,322 : INFO : topic #45 (0.020): 0.235*"less than 300 ratings" + 0.109*"Drama" + 0.061*"nudity (topless - notable)" + 0.036*"Comedy" + 0.035*"tim burton" + 0.023*"library" + 0.018*"michael moore" + 0.015*"blindfold" + 0.014*"depressing" + 0.011*"golden raspberry (worst actor)"
2021-08-24 02:42:15,323 : INFO : topic #37 (0.020): 0.185*"War" + 0.109*"world war ii" + 0.103*"Drama" + 0.047*"history" + 0.047*"war" + 0.035*"jim carrey" + 0.025*"Action" + 0.019*"70mm" + 0.014*"nazis" + 0.013*"mental illness"
2021-08-24 02:42:15,324 : INFO : topic #38 (0.020): 0.142*"drugs" + 0.037*"samuel l. jackson" + 0.035*"peter sellers" + 0.030*"ewan mcgregor" + 0.024*"divx" + 0.022*"poverty" + 0.020*"addiction" + 0.019*"hulu" + 0.018*"michael caine" + 0.017*"Drama"
2021-08-24 02:42:15,325 : INFO : topic diff=0.119826, rho=0.346261
2021-08-24 02:42:15,327 : INFO : PROGRESS: pass 2, at document #10000/10681
2021-08-24 02:42:15,579 : INFO : merging changes from 2000 documents into a model

2021-08-24 02:42:16,743 : INFO : topic #11 (0.020): 0.127*"religion" + 0.115*"keanu reeves" + 0.020*"dark" + 0.018*"reality tv" + 0.018*"courtesan" + 0.018*"slash" + 0.017*"slavery" + 0.017*"facebook rec" + 0.014*"class issues" + 0.012*"product placement"
2021-08-24 02:42:16,744 : INFO : topic diff=0.113857, rho=0.327201
2021-08-24 02:42:16,746 : INFO : PROGRESS: pass 3, at document #8000/10681
2021-08-24 02:42:16,986 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:42:17,013 : INFO : topic #20 (0.020): 0.086*"biography" + 0.060*"Drama" + 0.043*"biopic" + 0.042*"ingmar bergman" + 0.035*"ensemble cast" + 0.031*"julia roberts" + 0.030*"suicide" + 0.028*"corvallis library" + 0.027*"multiple storylines" + 0.025*"medieval"
2021-08-24 02:42:17,015 : INFO : topic #47 (0.020): 0.059*"serial killer" + 0.046*"psychology" + 0.045*"brad pitt" + 0.041*"edward norton" + 0.033*"heist" + 0.031*"hayao miyazaki" + 0.031*"steven spielberg" + 0.029*"crime" + 0.029

...

2021-08-24 02:42:55,939 : INFO : topic #18 (0.020): 0.082*"magic" + 0.079*"gay" + 0.062*"racism" + 0.044*"pg13" + 0.040*"Drama" + 0.027*"food" + 0.027*"social commentary" + 0.023*"homosexuality" + 0.020*"19th century" + 0.019*"denzel washington"
2021-08-24 02:42:55,940 : INFO : topic #19 (0.020): 0.062*"nudity (full frontal)" + 0.061*"drama" + 0.035*"Drama" + 0.034*"philip k. dick" + 0.028*"vietnam war" + 0.027*"rape" + 0.025*"vietnam" + 0.022*"very good" + 0.020*"to see" + 0.016*"slow"
2021-08-24 02:42:55,942 : INFO : topic diff=0.046241, rho=0.175844
2021-08-24 02:42:55,943 : INFO : PROGRESS: pass 26, at document #8000/10681
2021-08-24 02:42:56,180 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:42:56,207 : INFO : topic #2 (0.020): 0.287*"Documentary" + 0.104*"to see" + 0.064*"documentary" + 0.026*"mockumentary" + 0.025*"sexuality" + 0.023*"star trek" + 0.021*"dogs" + 0.021*"kevin smith" + 0.018*"movie to see" + 0.017*"in netflix queue"
2021

2021-08-24 02:42:57,341 : INFO : topic diff=0.023122, rho=0.173186
2021-08-24 02:42:57,343 : INFO : PROGRESS: pass 27, at document #6000/10681
2021-08-24 02:42:57,586 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:42:57,613 : INFO : topic #18 (0.020): 0.081*"magic" + 0.079*"gay" + 0.062*"racism" + 0.044*"pg13" + 0.040*"Drama" + 0.027*"food" + 0.027*"social commentary" + 0.023*"homosexuality" + 0.020*"19th century" + 0.019*"denzel washington"
2021-08-24 02:42:57,614 : INFO : topic #32 (0.020): 0.135*"johnny depp" + 0.067*"clint eastwood" + 0.059*"vhs" + 0.045*"jackie chan" + 0.037*"western" + 0.031*"kung fu" + 0.031*"leonardo dicaprio" + 0.030*"spaghetti western" + 0.029*"propaganda" + 0.029*"david lynch"
2021-08-24 02:42:57,615 : INFO : topic #26 (0.020): 0.042*"Drama" + 0.035*"james bond" + 0.024*"007" + 0.023*"bond" + 0.021*"mel gibson" + 0.021*"reflective" + 0.020*"atmospheric" + 0.019*"poignant" + 0.018*"lyrical" + 0.017*"bittersweet"
202

2021-08-24 02:42:59,028 : INFO : topic #7 (0.020): 0.113*"romance" + 0.065*"Romance" + 0.054*"chick flick" + 0.038*"boring" + 0.030*"girlie movie" + 0.027*"baseball" + 0.023*"Comedy" + 0.020*"love story" + 0.019*"whimsical" + 0.017*"wedding"
2021-08-24 02:42:59,029 : INFO : topic #5 (0.020): 0.292*"classic" + 0.062*"musical" + 0.045*"afi 100 (laughs)" + 0.034*"Musical" + 0.030*"national film registry" + 0.021*"afi 100" + 0.016*"70mm" + 0.015*"adapted from b'way" + 0.015*"john travolta" + 0.015*"breakthroughs"
2021-08-24 02:42:59,029 : INFO : topic #31 (0.020): 0.127*"zombies" + 0.074*"horror" + 0.072*"pg-13" + 0.039*"cult classic" + 0.025*"infidelity" + 0.025*"campy" + 0.024*"zombie" + 0.024*"joaquin phoenix" + 0.024*"sam raimi" + 0.020*"books"
2021-08-24 02:42:59,030 : INFO : topic diff=0.022572, rho=0.170646
2021-08-24 02:42:59,032 : INFO : PROGRESS: pass 28, at document #6000/10681
2021-08-24 02:42:59,270 : INFO : merging changes from 2000 documents into a model of 10681 documents
2

2021-08-24 02:43:00,446 : INFO : topic #17 (0.020): 0.330*"Sci-Fi" + 0.071*"Horror" + 0.068*"Action" + 0.044*"video game adaptation" + 0.027*"animals" + 0.025*"movie to see" + 0.022*"football" + 0.018*"futuristic" + 0.015*"milla jovovich" + 0.013*"g"
2021-08-24 02:43:00,447 : INFO : topic #34 (0.020): 0.074*"ghosts" + 0.051*"bill murray" + 0.044*"television" + 0.030*"courtroom" + 0.030*"courtroom drama" + 0.026*"secret service" + 0.021*"roman polanski" + 0.017*"18th century" + 0.017*"opera" + 0.017*"court"
2021-08-24 02:43:00,448 : INFO : topic diff=0.124178, rho=0.168215
2021-08-24 02:43:00,449 : INFO : PROGRESS: pass 29, at document #4000/10681
2021-08-24 02:43:00,687 : INFO : merging changes from 2000 documents into a model of 10681 documents
2021-08-24 02:43:00,714 : INFO : topic #31 (0.020): 0.127*"zombies" + 0.074*"horror" + 0.072*"pg-13" + 0.039*"cult classic" + 0.025*"infidelity" + 0.025*"campy" + 0.024*"zombie" + 0.024*"joaquin phoenix" + 0.024*"sam raimi" + 0.020*"books"
2021

2021-08-24 02:43:01,815 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel(num_terms=15261, num_topics=50, decay=0.5, chunksize=2000) in 51.72s', 'datetime': '2021-08-24T02:43:01.815087', 'gensim': '4.0.1', 'python': '3.7.8 (default, May  2 2021, 19:43:55) \n[Clang 10.0.1 (clang-1001.0.46.4)]', 'platform': 'Darwin-18.7.0-x86_64-i386-64bit', 'event': 'created'}


In [10]:
#  評価
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.004, Recall@K=0.012
